In [27]:
#Adrian Bozocea Datahon 5/12/2025


ICD_code, ICD_version,  Description
5845	9	Acute kidney failure with lesion of tubular necrosis
5846	9	Acute kidney failure with lesion of renal cortical necrosis
5847	9	Acute kidney failure with lesion of renal medullary [papillary] necrosis
5848	9	Acute kidney failure with other specified pathological lesion in kidney
5849	9	Acute kidney failure, unspecified
586     9   Renal failure, unspecified 

N17	    10	Acute kidney failure
N170	10	Acute kidney failure with tubular necrosis
N171	10	Acute kidney failure with acute cortical necrosis
N172	10	Acute kidney failure with medullary necrosis
N178	10	Other acute kidney failure
N179	10	Acute kidney failure, unspecified

R6521	10	Severe sepsis with septic shock
78552	9	Septic shock

66930	9	Acute kidney failure following labor and delivery, unspecified as to episode of care or not applicable
66932	9	Acute kidney failure following labor and delivery, delivered, with mention of postpartum complication
66934	9	Acute kidney failure following labor and delivery, postpartum condition or complication

O904	10	Postpartum acute kidney failure

In [ ]:
#imports
import pandas as pd
import pathlib
import numpy as np

#import data
filepath_dx_icd = r"C:\diagnoses_icd.csv.gz"
filepath_patients = r"C:\patients.csv.gz"
filepath_admissions = r"C:\admissions.csv.gz"

file_dx_icd = pathlib.Path(filepath_dx_icd)
file_patients = pathlib.Path(filepath_patients)
file_admissions = pathlib.Path(filepath_admissions)

df_dx_icd = pd.read_csv(filepath_dx_icd, compression = 'gzip')
df_patients = pd.read_csv(filepath_patients, compression = 'gzip')
df_admissions = pd.read_csv(filepath_admissions, compression = 'gzip')



export_file_dx_icd = r"C:\selected_patients_icd.xlsx"
export_file_dx_icd_demo = r"C:\selected_patients_icd_demo.xlsx"
export_file_dx_icd_demo_drgs = r"C:\selected_patients_icd_demo_drgs.csv"
export_file_dx_icd_demo_drgs_labs = r"C:\selected_patients_icd_demo_drgs_labs.csv"
export_file_dx_icd_demo_diabetes = r"C:\selected_patients_icd_demo_diabetes.csv"

export_file_dx_icd = pathlib.Path(export_file_dx_icd)
export_file_dx_icd_demo = pathlib.Path(export_file_dx_icd_demo)
export_file_dx_icd_demo_drgs = pathlib.Path(export_file_dx_icd_demo_drgs)
export_file_dx_icd_demo_drgs_labs = pathlib.Path(export_file_dx_icd_demo_drgs_labs)
export_file_dx_icd_demo_diabetes = pathlib.Path(export_file_dx_icd_demo_diabetes)



In [ ]:
#import emr
filepath_emar = r"C:\emar.csv.gz"
filepath_emar = pathlib.Path(filepath_emar)
df_emar = pd.read_csv(filepath_emar, compression = 'gzip')

In [ ]:
#import pharmacy
filepath_pharmacy = r"C:\pharmacy.csv.gz"
file_pharmacy = pathlib.Path(filepath_pharmacy)
df_pharmacy = pd.read_csv(file_pharmacy, compression = 'gzip')


In [ ]:
#import labevents
filepath_labevents = r"C:\labevents.csv.gz"
filepath_labevents = pathlib.Path(filepath_labevents)
df_labevents = pd.read_csv(filepath_labevents, compression = 'gzip')

In [ ]:
#selection functions

#check if icd code matches
def code_matches(code_str, target_code):
    target = str(target_code)
    #if alphabetic code (like E10), check for both exact match and variations
    if any(c.isalpha() for c in target):
        return code_str.startswith(target)
    else:
        #for period-containing codes like '250.4'
        if '.' in target:
            return code_str == target or code_str.replace('.', '') == target.replace('.', '')
        #for numeric codes
        return code_str == target

#creating filtering function for diagnoses
def create_code_mask(df, code_list):
    mask = pd.Series(False, index=df.index)
    for code in code_list:
        code_mask = df['icd_code_str'].apply(lambda x: code_matches(x, code))
        mask = mask | code_mask
    return mask


In [ ]:
#selection of patients based on ICD code 

icd_inclusion = [5845, 5846, 5847, 5848, 5849, 586, 'N17', 'N170', 'N171' , 'N172' , 'N178', 'N179' , 'R6521', 78552, 66930, 66932, 66934, 'O904']

df_dx_icd['icd_code_str'] = df_dx_icd['icd_code'].astype(str)

aki_mask = create_code_mask(df_dx_icd, icd_inclusion)
selected_patients_icd = df_dx_icd[aki_mask].drop(columns=['icd_code_str'])

selected_patients_icd.to_excel(export_file_dx_icd, index=False)

In [ ]:
#combining selected patients with their demographic data

selected_patients_icd_demo = pd.merge(
    selected_patients_icd,
    df_patients,
    how='inner',
    on='subject_id'
)

admission_columns = ['subject_id', 'hadm_id', 'admission_type', 'admission_location', 
                      'insurance', 'language', 'marital_status', 'race']
admission_subset = df_admissions[admission_columns]


selected_patients_icd_demo = pd.merge(
    selected_patients_icd_demo,
    admission_subset,
    how='left',
    on=['subject_id', 'hadm_id']
)

selected_patients_icd_demo.to_excel(export_file_dx_icd_demo, index=False)

In [ ]:
#combining selected patients with their medication data

pharmacy_columns = ['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id' 'starttime', 'stoptime', 'medication' , 'proc_type',
                   'status' , 'route' , 'frequency' , 'disp_sched' , 'doses_per_24_hrs' , 'duration_interval']
pharmacy_subset = df_pharmacy[pharmacy_columns]

pharmacy_subset.rename(columns = {'starttime':'starttime_pharm' , 'stoptime':'stoptime_pharm', 'proc_type':'proc_type_pharm',
                                  'stats':'status_pharm' , 'route':'route_pharm' , 'frequency':'frequency_pharm',
                                  'disp_sched':'disp_sched_pharm', 'doses_per_24_hrs':'doses_per_24_hrs_pharm', 'duration_interval':'duration_interval_pharm'} , inplace=True)

emar_columns = ['subject_id' ,'hadm_id', 'emar_id', 'emar_seq' , 'poe_id' , 'pharmacy_id' , 'charttime' , 'medication' , 'event_txt']
emar_subset = df_emar[emar_columns]

emar_subset.rename(columns = {'charttime':'charttime_emar', 'event_txt':'event_txt_emar'} , inplace=True)

selected_patients_icd_demo_drgs = pd.merge(
    selected_patients_icd_demo,
    pharmacy_subset,
    how='left',
    on=['subject_id', 'hadm_id']
)

selected_patients_icd_demo_drgs = pd.merge(
    selected_patients_icd_demo_drgs,
    emar_subset,
    how='left',
    on=['subject_id', 'hadm_id']
)

selected_patients_icd_demo_drgs.to_csv(export_file_dx_icd_demo_drgs, index=False)

Codes for labs

from d_labitems -

itemid,  label
50920	Estimated GFR (MDRD equation)
51770	MDRDgfr
52026	Estimated GFR (MDRD equation)
53161	Estimated GFR (CKD- EPI Refit)
53180	Estimated GFR (CKD- EPI 2021)



In [ ]:
#combining patients with their lab data

lab_codes = [50920, 51770, 52026, 53161, 53180]

selected_lab_events = df_labevents[df_labevents['itemid'].astype(str).isin([str(code) for code in icd_inclusion])]

lab_event_columns = ['labevent_id'	,'subject_id'	,'hadm_id'	,'specimen_id' ,'order_provider_id' ,'value' ,'valuenum' ,'valueuom'	
                     'ref_range_lower'	,'ref_range_upper'	,'flag'	,'priority'	,'comments']

selected_lab_events = selected_lab_events[lab_event_columns]
selected_lab_events.rename(columns = {'itemid':'itemid_lab' ,
                                      'specimen_id':'specimen_id_lab',
                                      'charttime':'charttime_lab',
                                      'storetime':'storetime_lab',
                                      'value':'value_lab',
                                      'valuenum':'valuenum_lab', 
                                      'valueuom':'valueuom_lab',
                                      'ref_range_lower':'ref_range_lower_lab',
                                      'ref_range_upper':'ref_range_upper_lab',
                                      'flag':'flag_lab',
                                      'priority':'priority_lab',
                                      'comments':'comments_lab'}, inplace=True)


selected_patients_icd_demo_drgs_labs = pd.merge(
    selected_patients_icd_demo_drgs,
    selected_lab_events,
    how='left',
    on=['subject_id', 'hadm_id']
)

selected_patients_icd_demo_drgs_labs.to_csv(export_file_dx_icd_demo_drgs_labs, index=False)

Codes for clinical factors 

icd_code, icd_version, description
25000	9	Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled
25001	9	Diabetes mellitus without mention of complication, type I [juvenile type], not stated as uncontrolled
25002	9	Diabetes mellitus without mention of complication, type II or unspecified type, uncontrolled
25003	9	Diabetes mellitus without mention of complication, type I [juvenile type], uncontrolled
25010	9	Diabetes with ketoacidosis, type II or unspecified type, not stated as uncontrolled
25011	9	Diabetes with ketoacidosis, type I [juvenile type], not stated as uncontrolled
25012	9	Diabetes with ketoacidosis, type II or unspecified type, uncontrolled
25013	9	Diabetes with ketoacidosis, type I [juvenile type], uncontrolled
25020	9	Diabetes with hyperosmolarity, type II or unspecified type, not stated as uncontrolled
25021	9	Diabetes with hyperosmolarity, type I [juvenile type], not stated as uncontrolled
25022	9	Diabetes with hyperosmolarity, type II or unspecified type, uncontrolled
25023	9	Diabetes with hyperosmolarity, type I [juvenile type], uncontrolled
25030	9	Diabetes with other coma, type II or unspecified type, not stated as uncontrolled
25031	9	Diabetes with other coma, type I [juvenile type], not stated as uncontrolled
25032	9	Diabetes with other coma, type II or unspecified type, uncontrolled
25033	9	Diabetes with other coma, type I [juvenile type], uncontrolled

25040	9	Diabetes with renal manifestations, type II or unspecified type, not stated as uncontrolled
25041	9	Diabetes with renal manifestations, type I [juvenile type], not stated as uncontrolled
25042	9	Diabetes with renal manifestations, type II or unspecified type, uncontrolled
25043	9	Diabetes with renal manifestations, type I [juvenile type], uncontrolled

25050	9	Diabetes with ophthalmic manifestations, type II or unspecified type, not stated as uncontrolled
25051	9	Diabetes with ophthalmic manifestations, type I [juvenile type], not stated as uncontrolled
25052	9	Diabetes with ophthalmic manifestations, type II or unspecified type, uncontrolled
25053	9	Diabetes with ophthalmic manifestations, type I [juvenile type], uncontrolled
25060	9	Diabetes with neurological manifestations, type II or unspecified type, not stated as uncontrolled
25061	9	Diabetes with neurological manifestations, type I [juvenile type], not stated as uncontrolled
25062	9	Diabetes with neurological manifestations, type II or unspecified type, uncontrolled
25063	9	Diabetes with neurological manifestations, type I [juvenile type], uncontrolled
25070	9	Diabetes with peripheral circulatory disorders, type II or unspecified type, not stated as uncontrolled
25071	9	Diabetes with peripheral circulatory disorders, type I [juvenile type], not stated as uncontrolled
25072	9	Diabetes with peripheral circulatory disorders, type II or unspecified type, uncontrolled
25073	9	Diabetes with peripheral circulatory disorders, type I [juvenile type], uncontrolled
25080	9	Diabetes with other specified manifestations, type II or unspecified type, not stated as uncontrolled
25081	9	Diabetes with other specified manifestations, type I [juvenile type], not stated as uncontrolled
25082	9	Diabetes with other specified manifestations, type II or unspecified type, uncontrolled
25083	9	Diabetes with other specified manifestations, type I [juvenile type], uncontrolled
25090	9	Diabetes with unspecified complication, type II or unspecified type, not stated as uncontrolled
25091	9	Diabetes with unspecified complication, type I [juvenile type], not stated as uncontrolled
25092	9	Diabetes with unspecified complication, type II or unspecified type, uncontrolled
25093	9	Diabetes with unspecified complication, type I [juvenile type], uncontrolled

2535	9	Diabetes insipidus

E10     10  Type 1 Diabetes mellitus
E10#    10  Type 1 Diabetes mellitus with complications

E102	10	Type 1 diabetes mellitus with kidney complications
E1021	10	Type 1 diabetes mellitus with diabetic nephropathy
E1022	10	Type 1 diabetes mellitus with diabetic chronic kidney disease
E1029	10	Type 1 diabetes mellitus with other diabetic kidney complication

E11     10  Type 2 Diabetes mellitus 
E11#    10  Type 2 Diabetes meelitus with complications 

E112	10	Type 2 diabetes mellitus with kidney complications
E1121	10	Type 2 diabetes mellitus with diabetic nephropathy
E1122	10	Type 2 diabetes mellitus with diabetic chronic kidney disease
E1129	10	Type 2 diabetes mellitus with other diabetic kidney complication

E13     10  Other specified diabetes mellitus 
E13#    10  Other specified diabetes mellitus with complications

E132	10	Other specified diabetes mellitus with kidney complications
E1321	10	Other specified diabetes mellitus with diabetic nephropathy
E1322	10	Other specified diabetes mellitus with diabetic chronic kidney disease
E1329	10	Other specified diabetes mellitus with other diabetic kidney complication

E232	10	Diabetes insipidus

N251	10	Nephrogenic diabetes insipidus




In [ ]:
#Grading for clinical factors 

general_diabetes_codes = [25000, 25001,25002, 25003, 25010, 25011, 25012, 25013, 25020, 25021, 25022, 25023, 25030, 25031, 25032, 25033, 25040,
25041, 25042, 25043, 25050, 25051, 25052, 25053, 25060, 25061, 25062, 25063, 25070, 25071, 25072, 25073, 25080, 25081, 25082, 25083, 25090, 25091, 25092, 25093, 2535,
'E10', 'E102', 'E1021', 'E1022', 'E1029', 'E11', 'E112', 'E1121', 'E1122', 'E1129', 'E13', 'E132','E1321','E1322','E1329','E232','N251']

diabetes_renal = [25040, 25041, 25042, 25043, 'E102', 'E1021', 'E1022', 'E1029', 'E112', 'E1121', 'E1122', 'E1129', 'E132', 'E1321', 'E1322', 'E1329', 'E232']

general_diabetes_mask = create_code_mask(df_dx_icd, general_diabetes_codes)
diabetes_renal_mask = create_code_mask(df_dx_icd, diabetes_renal)

selected_subject_ids = selected_patients_icd_demo['subject_id'].unique()

diabetes_flags = pd.DataFrame({'subject_id': selected_subject_ids})
subjects_with_diabetes =  set(df_dx_icd[general_diabetes_mask]['subject_id'].unique())
subjects_with_diabetes_renal = set(df_dx_icd[diabetes_renal_mask]['subject_id'].unique())

diabetes_flags['diabetes'] = diabetes_flags['subject_id'].apply(
    lambda x:1 if x in subjects_with_diabetes else 0
)

diabetes_flags['diabetes_renal'] = diabetes_flags['subject_id'].apply(
    lambda x:1 if x in subjects_with_diabetes_renal else 0
)

selected_patients_icd_demo_diabetes = pd.merge(
    selected_patients_icd_demo,
    diabetes_flags,
    how='left',
    on=['subject_id', 'hadmn_id']
)

selected_patients_icd_demo_diabetes.to_csv(export_file_dx_icd_demo_diabetes, index=False)

